# Experiment 4. Обучение модели с использованием PyTorch

## Цель эксперимента

Цель данного эксперимента заключается в оценке влияния обработки пропусков на производительность модели машинного обучения. Мы будем использовать `SimpleImputer` для заполнения пропусков в данных, а затем обучим простую нейронную сеть с использованием библиотеки PyTorch.

## Описание данных

Данные загружаются из файла `processed_application_train.csv`, который содержит информацию о заявках. Целевая переменная обозначена как `target`, и мы будем использовать остальные столбцы в качестве признаков для обучения модели.

## Процесс эксперимента

1. **Загрузка данных**: Данные загружаются из CSV файла с использованием библиотеки pandas.

2. **Обработка пропусков**: 
   - Для заполнения пропусков в данных используется `SimpleImputer` с стратегией `most_frequent`, что позволяет заменить пропуски наиболее частым значением в каждом столбце.
   
3. **Разделение данных**: Данные разделяются на обучающую и тестовую выборки с использованием `train_test_split` из библиотеки scikit-learn.

4. **Стандартизация данных**: Признаки стандартизируются с помощью `StandardScaler`, чтобы привести их к одному масштабу.

5. **Создание тензоров**: Данные преобразуются в тензоры PyTorch для дальнейшего использования в модели.

6. **Определение модели**: Создается простая нейронная сеть с двумя скрытыми слоями и выходным слоем с активацией сигмоид для бинарной классификации.

7. **Обучение модели**: Модель обучается на обучающей выборке с использованием бинарной кросс-энтропии в качестве функции потерь и Adam в качестве оптимизатора.

8. **Оценка модели**: После обучения модель оценивается на тестовой выборке, и вычисляется точность предсказаний.

## Результаты

В результате эксперимента мы получим значение точности модели на тестовой выборке, что позволит оценить, насколько эффективно заполнение пропусков повлияло на производительность модели.

## Заключение

Данный эксперимент поможет понять, как обработка пропусков в данных влияет на качество предсказаний модели. Результаты могут быть использованы для дальнейших улучшений в предобработке данных и выборе моделей.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from torch.utils.data import DataLoader, TensorDataset

from utils.roc_auc import plot_roc_curve

In [3]:
target_column = 'TARGET'

In [4]:
data = pd.read_csv("./data/processed_data/processed_application_train.csv", index_col=0)

In [5]:
# Предположим, что 'target' - это целевая переменная
X = data.drop(columns=[target_column]).values
y = data[target_column].values

# Обработка пропусков с помощью SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
X_imputed = imputer.fit_transform(X)

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.3, random_state=42)

# Стандартизация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 400)
        self.prelu1 = nn.PReLU()
        self.dropout1 = nn.Dropout(0.3)

        self.fc2 = nn.Linear(400, 160)
        self.prelu2 = nn.PReLU()
        self.bn2 = nn.BatchNorm1d(160)
        self.dropout2 = nn.Dropout(0.3)

        self.fc3 = nn.Linear(160, 64)
        self.prelu3 = nn.PReLU()
        self.bn3 = nn.BatchNorm1d(64)
        self.dropout3 = nn.Dropout(0.3)

        self.fc4 = nn.Linear(64, 26)
        self.prelu4 = nn.PReLU()
        self.bn4 = nn.BatchNorm1d(26)
        self.dropout4 = nn.Dropout(0.3)

        self.fc5 = nn.Linear(26, 12)
        self.prelu5 = nn.PReLU()
        self.bn5 = nn.BatchNorm1d(12)
        self.dropout5 = nn.Dropout(0.3)

        self.fc6 = nn.Linear(12, 1)

    def forward(self, x):
        x = self.dropout1(self.prelu1(self.fc1(x)))
        x = self.dropout2(self.bn2(self.prelu2(self.fc2(x))))
        x = self.dropout3(self.bn3(self.prelu3(self.fc3(x))))
        x = self.dropout4(self.bn4(self.prelu4(self.fc4(x))))
        x = self.dropout5(self.bn5(self.prelu5(self.fc5(x))))
        x = torch.sigmoid(self.fc6(x))
        return x

In [9]:
# Создание DataLoader
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train).view(-1, 1) 
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


# Инициализация модели, функции потерь и оптимизатора
model = SimpleNN()
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Divice is {device}")
model.to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0002)

# Обучение модели
num_epochs = 30
losses = [] 

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs)  
        loss = criterion(outputs, labels) 
        loss.backward()
        optimizer.step()

    losses.append(loss.item())
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Оценка модели
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_pred = (y_pred > 0.5).float()


# Вывод значений потерь
plt.plot(range(1, num_epochs + 1), losses, marker='o')
plt.title('Loss over epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid()
plt.show()

Divice is mps
Epoch [1/30], Loss: 0.2951
Epoch [2/30], Loss: 0.2122
Epoch [3/30], Loss: 0.1557
Epoch [4/30], Loss: 0.3950
Epoch [5/30], Loss: 0.3897
Epoch [6/30], Loss: 0.1395


Нейросеть не будем обучать в несколько итераций для проведения т-теста, так как это требует значительных вычислительных ресурсов и времени